<a href="https://colab.research.google.com/github/backeung/KoBERT-multi-classification-model/blob/main/KoBERT_B%EC%82%AC%EC%A0%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   KoBERT 다중분류모델(기존): https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab
*   기존 KoBERT 코드: https://github.com/Seolini/KoBERT_Korean_multi_classification/blob/main/KoBERT_%ED%95%9C%EA%B5%AD%EC%96%B4_7%EA%B0%9C%EA%B0%90%EC%A0%95_%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98.ipynb
*   ver2: https://github.com/SKTBrain/KoBERT/issues/67#L55


*   SKT Korean BERT: https://reposhub.com/python/deep-learning/SKTBrain-KoBERT.html

*   get_kobert_model(): https://github.com/monologg/KoBERT-Transformers
* 코드수정1: https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf
* 코드수정2: https://github.com/SKTBrain/KoBERT/issues/68

**1. Colab 환경설정**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/Shareddrives/종합설계2/Colab Notebooks/KoBERT'

/content/drive/Shareddrives/종합설계2/Colab Notebooks/KoBERT


In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git

fatal: destination path 'KoBERT' already exists and is not an empty directory.


In [ ]:
cd 'KoBERT'

/content/drive/Shareddrives/종합설계2/Colab Notebooks/KoBERT/KoBERT


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 46.9 MB 1.9 MB/s 
     |████████████████████████████████| 344 kB 37.8 MB/s 
     |████████████████████████████████| 1.2 MB 33.8 MB/s 
     |████████████████████████████████| 4.8 MB 42.6 MB/s 
     |████████████████████████████████| 1.3 MB 33.0 MB/s 
     |████████████████████████████████| 1.1 MB 39.7 MB/s 
     |████████████████████████████████| 895 kB 16.0 MB/s 
     |████████████████████████████████| 2.9 MB 26.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595732 sha256=334e056ade7429333e015a2b2ee6feaa47156eeb58f96894bb75ce35dfe7c417
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
!pip install .

Processing /content/drive/Shareddrives/종합설계2/Colab Notebooks/KoBERT/KoBERT
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12780 sha256=befce767f1dc572d760685e6ee3c0d61635cc298a6640905c6e62e5dbc2b9138
  Stored in directory: /tmp/pip-ephem-wheel-cache-p6gpmimp/wheels/1a/58/1f/6cfd2b2e6f63f207a90ec7b093d471d28a4f33de7c0f3af5fb
Successfully built kobert


In [ ]:
predefined_args = {
        'attention_cell': 'multi_head',
        'num_layers': 12,
        'units': 768,
        'hidden_size': 3072,
        'max_length': 512,
        'num_heads': 12,
        'scaled': True,
        'dropout': 0.1,
        'use_residual': True,
        'embed_size': 768,
        'embed_dropout': 0.1,
        'token_type_vocab_size': 2,
        'word_embed': None,
    }

In [ ]:
!pip install torch==1.7.0

     |████████████████████████████████| 776.7 MB 4.4 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.


In [ ]:
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#참고: 코드수정1
import torch
from kobert.pytorch_kobert import get_kobert_model, get_pytorch_kobert_model
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-up_m1brk/kobert-tokenizer_1206c7f981f349588da65a0dc42619e9
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-up_m1brk/kobert-tokenizer_1206c7f981f349588da65a0dc42619e9
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4682 sha256=420b247054dd3db96ec4637a928dbcf4cdd70b0c3e564f7567667e9ea48fffc3
  Stored in directory: /tmp/pip-ephem-wheel-cache-tkdkbrfs/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

get_kobert


Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

get_kobert


In [ ]:
print(bertmodel, vocab)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )

https://github.com/SKTBrain/KoBERT/issues/68
https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf

**2. 데이터셋전처리**

In [ ]:
#데이터 불러오기
import pandas as pd
chatbot_data = pd.read_excel('/content/drive/Shareddrives/종합설계2/Dataset/감성대화말뭉치(원시데이터)_Training.xlsx')

In [ ]:
chatbot_data.rename(columns= {"감정_대분류": "Emotion", "사람문장1" : "Sentence"}, inplace= True)


In [ ]:
df= chatbot_data.sample(n= 38594)


In [ ]:
df = df[['Sentence', 'Emotion']]
df

,Sentence,Emotion
54394,나 너무 친구때문에 당황스러워.,1
40158,가족이랑 놀러 왔는데 길을 잃었어.,1
70760,요즘 들어 부쩍 내가 늦깎이라는 생각이 드네. 나 자신이 황새 따라가는 뱁새처럼 한...,1
65720,남편이 요새 은퇴하면 집을 팔고 시골로 이사하자고 매일 같이 졸라서 골치가 아파.,2
12778,나 고민이 있어. 진이가 연숙이랑 자꾸 놀지 말래.,3
...,...,...
72437,건강이 걱정스러워.,3
29101,아프면 툴툴대게 되는 거 같아. 본인은 물론이지만 가족이 아프면 더.,2
36626,오늘 회사에서 승진 시험을 봤는데 떨어졌어.,2
37215,내가 따돌림 받는다고 친했던 친구들이 나랑 안논대.,4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38594 entries, 54394 to 45737
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  38594 non-null  object
 1   Emotion   38594 non-null  object
dtypes: object(2)
memory usage: 904.5+ KB


In [ ]:
#NaN값을 가지는 컬럼들을 없애고 Emotion컬럼에 7가지의 감정 단어들을 숫자로 변경
df.loc[(df['Emotion'] == "기쁨"), 'Emotion'] = 0  #기쁨 => 0
df.loc[(df['Emotion'] == "당황"), 'Emotion'] = 1  #당황 => 1
df.loc[(df['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
df.loc[(df['Emotion'] == "불안"), 'Emotion'] = 3  #불안 => 3
df.loc[(df['Emotion'] == "상처"), 'Emotion'] = 4  #상처 => 4
df.loc[(df['Emotion'] == "슬픔"), 'Emotion'] = 5  #슬픔 => 5

In [ ]:
#data_list = ('문장', '감정')
data_list = []
for q, label in zip(df['Sentence'], df['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(type(data_list))

<class 'list'>


In [ ]:
data_list

[['나 너무 친구때문에 당황스러워.', '1'],
 ['가족이랑 놀러 왔는데 길을 잃었어.', '1'],
 ['요즘 들어 부쩍 내가 늦깎이라는 생각이 드네. 나 자신이 황새 따라가는 뱁새처럼 한심하게만 여겨지니 어떡하지?', '1'],
 ['남편이 요새 은퇴하면 집을 팔고 시골로 이사하자고 매일 같이 졸라서 골치가 아파.', '2'],
 ['나 고민이 있어. 진이가 연숙이랑 자꾸 놀지 말래.', '3'],
 ['십 년 전에 새집을 지었는데 아직도 대출금을 갚고 있어.', '3'],
 ['나는 너무 화가 나고 속상해. 나의 아내가 다른 남자를 몰래 만나고 있는 것 같아.', '4'],
 ['이겼다! 정말 쉽지 않은 경기였는데. 오늘도 내가 응원하는 야구팀이 승리를 차지했어.', '0'],
 ['당신은 어떤 생각인가요?', '0'],
 ['내 친구랑 또 싸웠어.', '2'],
 ['난 그동안 대인관계를 잘해온 것 같아서 감사해.', '0'],
 ['친구에게 늦지 말라고 말해봐야겠어.', '2'],
 ['모두가 같은 길로 가는데 나만 다른 길을 가니 버려진 기분이야.', '4'],
 ['눈에 염증이 생겨서 안과에 가니 칼로 찢어서 치료를 해야한데 그래서 많이 놀랬어.', '4'],
 ['지금까지 열심히 살아왔는데 내가 암이라니 너무 화나.', '2'],
 ['어제부터 옆구리에 통증이 있어서 암이 전이된 건 아닌지 불안해.', '3'],
 ['나이가 드니 어리석은 행동을 많이 보이게 돼.', '1'],
 ['휴 다행이야 내 친구보다 좋은 회사에서 오늘 합격 통보를 받았어', '0'],
 ['해결이 안 된다고 징징거리는데 그냥 어쩔 수 없다고 하고 거절했어.', '5'],
 ['가족들과 대화도 잘 통해서인지 집안 분위기가 편안해.', '0'],
 ['직장을 그만둬서 수입이 없는데 아들이 자꾸 경제적인 지원을 바라서 상처받았어.', '4'],
 ['아들이 결혼 적령기인데 스타일에도 신경 좀 쓰면 좋을 텐데.', '1'],
 ['내 자신이 부끄러워.', '1'],
 ['

**3. Train Data & Test Data**

In [ ]:
#train:test = 7:3
from sklearn.model_selection import train_test_split
                                                         
train,test = train_test_split(data_list, test_size=0.3, random_state=1234)

In [ ]:
print(len(train)) #27015
print(len(test)) #11579

27015
11579


In [ ]:
test

[['나는 내 지금 건강상태에 만족해.', '0'],
 ['부하 직원들이 나만 빼돌리고 이차 회식 자리를 갔어.', '4'],
 ['아들이 양말과 속옷을 뒤집어 빨래통에 넣어. 한두 번도 아니고 고쳐지지 않아.', '2'],
 ['김 씨는 고혈압이라 말로는 기름진 음식 안 먹는다고 하면서 노인정에서 전 먹을 때는 가장 많이 먹는데 한심해.', '1'],
 ['노인정 사람들 때문에 스트레스를 받아.', '3'],
 ['오늘 직원들 앞에서 발표를 하다가 사래가 걸렸어. 정말 당황스러웠어.', '1'],
 ['나 어떡해? 나 결혼은 할 수 있을까? 아니 지금 누굴 만날 수 있을까?', '5'],
 ['딸이 오늘도 규칙을 어기고 외박을 했어.', '2'],
 ['이제 병이 다 나아서 치료 안 받아도 된다고 하니 너무 편안한 기분이야.', '0'],
 ['정년이 얼마남지 않았는데 노후 준비를 하나도 안해놓아서 걱정이야.', '3'],
 ['나는 비록 다리를 절지만 성실하게 살아서 재정에는 문제가 없어.', '0'],
 ['난 돈도 없고 취업도 못 하고 참 우울하다.', '4'],
 ['남편하고 애들 사이가 점점 나빠지는 거 같아. 남편은 좀 권위적이고 애들은 개성이 강한 편이야.', '3'],
 ['사무실에서 같이 근무하는 동료와 말다툼을 했는데 불편해서 걱정이야.', '5'],
 ['과거에 보증을 서지 말았었어야 하는데.', '5'],
 ['결혼할 나이가 되니까 예전에 집안도 좋고 성격도 좋던 여자친구와 헤어진게 후회 돼.', '5'],
 ['젊었을 때 부부싸움 하면서 내가 부인에게 손찌검을 한 적이 있었어.', '1'],
 ['요즘들어  눈물이나.', '5'],
 ['돈이 없으면 버려지는게 이 나이대 공통인가봐.', '2'],
 ['오늘 엄마가 아파서 가족 다 같이 병원에 갔다 왔어. 엄마가 중환자실에서 아파하고 있는 모습을 보니까 너무 눈물이 나.', '5'],
 ['직급이 낮은 사원이 나보다 일을 더 잘하는 것 같아 스트레스 받아.', '1'],
 ['이제 고등

**4.KoBERT 입력 데이터로 만들기**

In [ ]:
#BERTDataset 클래스 생성
#각 데이터가 KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 등 실시
#BERTSentenceTransform 이라는 모듈사용-> 토큰화, 패딩 이루어짐
#참고: 코드수정2 ->class에서 def init안에 vocab추가 
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
#하이퍼 파라미터 조정
#사실 파라미터의 값은 정답이 없고 가장 좋은 성능을 내는 값을 찾아가는 것이 좋음
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
#string error 해결 -> vocab 추가
tok=tokenizer.tokenize
data_train = BERTDataset(train, 0, 1, tok,vocab, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
#torch 형식의 dataset을 만들기
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

**5. KoBERT 학습모델 만들기**

In [ ]:
#KoBERT학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

**6. KoBERT모델 학습시키기**

In [ ]:
#KoBERT 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/423 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9383244514465332 train acc 0.125
epoch 1 batch id 201 loss 0.8121947646141052 train acc 0.5939054726368159
epoch 1 batch id 401 loss 0.8035557866096497 train acc 0.6818500623441397
epoch 1 train acc 0.6864604441067207


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/181 [00:00<?, ?it/s]

...

epoch 9 test acc 0.7592061522614476


  0%|          | 0/423 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.07051447033882141 train acc 0.984375
epoch 10 batch id 201 loss 0.1489565521478653 train acc 0.9849191542288557
epoch 10 batch id 401 loss 0.03928288817405701 train acc 0.9832060473815462
epoch 10 train acc 0.9832299054373522


  0%|          | 0/181 [00:00<?, ?it/s]

epoch 10 test acc 0.7601557378968068


In [ ]:
#학습시킨 모델 파일로 저장
import joblib
joblib.dump(model, './kobert_trained_model_ver5.pkl')

['./kobert_trained_model_ver4.pkl']

In [ ]:
#print(model)

In [ ]:
#학습시킨 모델 파일 불러오기
#import joblib
#loaded_model = joblib.load('kobert_trained_model_ver4.pkl')